In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy import sparse

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.linear_model import LinearRegression
from sklearn import svm

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.linear_model import Ridge

import gensim
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

import pickle

from sklearn.tree import DecisionTreeRegressor




## Loading training Data

In [2]:
train = pd.read_csv('D://Project//NLP//dataset//train.csv',  usecols=['target', 'comment_text'])

## Target Variable

In [3]:
y = train['target']
del train['target']

## Train- Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split( train, y, test_size=0.20, random_state=0)

# Data Preprocessing

In [5]:
def text_prepare(text): ### The function will take in text and lower case it remove the stopwords, symbols and return it
    
    STOPWORDS = set(stopwords.words('english'))

    text = text.lower()                     
    text = re.compile('[/(){}\[\]\|@,;]').sub(' ', text)    
    text = re.compile('[^0-9a-z #+_]').sub('', text)
    pattern = re.compile(r'\b(' + r'|'.join(STOPWORDS) + r')\b\s*') 
    text = pattern.sub('', text)    
    text = re.sub(' +', ' ', text)
        
    return text

# Handle missing Comments by unknown Value and Pre-process the data

In [6]:
COMMENT = 'comment_text'
X_train[COMMENT].fillna("unknown", inplace=True)
X_test[COMMENT].fillna("unknown", inplace=True)
X_train[COMMENT] = [text_prepare(COMMENT) for COMMENT in X_train[COMMENT]]
X_test[COMMENT] = [text_prepare(COMMENT) for COMMENT in X_test[COMMENT]]


C:\Users\manis\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\manis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\manis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

# Create Backup of the train test split

In [7]:
back_up_train = X_train
back_up_test = X_test

## TF_IDF Vectorizer

In [8]:
def tf_idf(X_train, X_test):
    vect_word = TfidfVectorizer(max_features=30000, lowercase=True, analyzer='word',stop_words= 'english',dtype=np.float32)
    tr_vect = vect_word.fit_transform(X_train['comment_text'])
    ts_vect = vect_word.transform(X_test['comment_text'])
    X = sparse.hstack([tr_vect])
    x_test = sparse.hstack([ts_vect])
    return X, x_test, vect_word

In [10]:
X, x_test ,vect_word= tf_idf(X_train, X_test)
pickle.dump(vect_word, open("TF_IDF_Transformer", 'wb'))

# Using Under- Sampling and Google word-to-vec

In [11]:
google_word_vec = gensim.models.KeyedVectors.load_word2vec_format("C:/Users/manis/Downloads/GoogleNews-vectors-negative300.bin.gz", binary = True )


In [12]:
more = 0
less = 0
y1 = list(y_train)
for i in range(len(y_train)):
    if ((y1)[i] >= 0.4):
        more += 1
    else:
        less += 1


In [13]:
X_train = X_train.reset_index()

In [14]:
newy = []
v = min(more, less)
a = 0
b = 0
newx = []
for i in range(len(y_train)):
    if ((y1)[i] >= 0.4 and a < v):
        a += 1
        newx.append(X_train['comment_text'][i])
        newy.append((y1)[i])
    if (y1[i] < 0.4 and b < v):
        b += 1
        newx.append(X_train['comment_text'][i])
        newy.append((y1)[i])
    if (a == v and b == v):
        break

In [15]:
def wordToVec(sentences, google_word_vec):
    
    sent_vocab = []
    c = 0
    word2vect = google_word_vec.vocab.keys()  
    for sentence in sentences:
        if (c == -1):
            break
        word_vocab = []
        words = sentence.split()
        for word in words:
            if word in word2vect:
                word_vocab.append( google_word_vec[word] )
        word_vocab = np.mean( word_vocab, axis = 0 )
#         print(c)
#         if (c == 397):
#             print("adfa")
        if (word_vocab.shape == ()):
            word_vocab = [0]*300
        sent_vocab.append(list(word_vocab))
        c += 1
    #data = [np.array( sent_vocab ), list(sentences)]
    return (sent_vocab)


In [18]:
x_train_word = wordToVec(newx,google_word_vec) 


In [19]:
x_test_word = wordToVec(X_test['comment_text'],google_word_vec) 


In [16]:
def tf_idf_sampled(X_train, X_test):
    vect_word = TfidfVectorizer(max_features=30000, lowercase=True, analyzer='word',stop_words= 'english',dtype=np.float32)
    tr_vect = vect_word.fit_transform(X_train)
    ts_vect = vect_word.transform(X_test['comment_text'])
    X = sparse.hstack([tr_vect])
    x_test = sparse.hstack([ts_vect])
    return X, x_test, vect_word

In [17]:
X_tfidf, x_test_tfidf, vect_word = tf_idf_sampled(newx, X_test)
pickle.dump(vect_word, open("TF_IDF_Transformer_Sampled", 'wb'))

In [31]:
def linearRegression(X, y_train, x_test, y_test, filename):
    lr = LinearRegression()
    lr.fit(X,y_train)
    #cv_score.append(lr.score)
    pred_linear = lr.predict(x_test)
    print("Mean Square Error for Linear Regression is : ", mean_squared_error(y_test, pred_linear))
    pickle.dump(lr, open(filename, 'wb'))

In [32]:
def lassoRegression(X, y_train, x_test, y_test,filename):
    clf = linear_model.Lasso(alpha=0.1)
    clf.fit(X,y_train)
    pred_lasso = clf.predict(x_test)
    print("Mean Square Error for Lasso Regression is : ", mean_squared_error(y_test, pred_lasso))
    pickle.dump(clf, open(filename, 'wb'))

In [33]:
def RidgeRegression(X, y_train, x_test, y_test,filename):
    clf = Ridge(alpha=1.0)
    clf.fit(X,y_train)
    pred_ridge = clf.predict(x_test)
    print("Mean Square Error for Ridge Regression is : ",mean_squared_error(y_test, pred_ridge))
    pickle.dump(clf, open(filename, 'wb'))

In [34]:
def RandomForest(X, y_train, x_test, y_test,filename):
    regr = RandomForestRegressor(max_depth=20, random_state=0)
    regr.fit(X, y_train)  

    pred_randomforest = regr.predict(x_test)
    print("Mean Square Error for Random Forest Regression is : ",mean_squared_error(y_test, pred_randomforest))
    pickle.dump(regr, open(filename, 'wb'))

In [46]:
def Decisiontree(X, y_train, x_test, y_test,filename):
    regr = DecisionTreeRegressor(max_depth=20, random_state=0)
    regr.fit(X, y_train)  

    pred_randomforest = regr.predict(x_test)
    print("Mean Square Error for DecisionTree Regression is : ",mean_squared_error(y_test, pred_randomforest))
    pickle.dump(regr, open(filename, 'wb'))

# Unsampled Results

In [35]:
linearRegression(X, y_train, x_test, y_test,"linear_Regression_TF_IDF.model")
lassoRegression(X, y_train, x_test, y_test,"lasso_Regression_TF_IDF.model")
RidgeRegression(X, y_train, x_test, y_test,"Ridge_Regression_TF_IDF.model")
#RandomForest(X, y_train, x_test, y_test,"RandomForest_Regression_TF_IDF.model")

Mean Square Error for Linear Regression is :  0.01871687792612595
Mean Square Error for Lasso Regression is :  0.039165215832144995
Mean Square Error for Ridge Regression is :  0.018140464198209232


KeyboardInterrupt: 

# Sampled Results

In [40]:
linearRegression(X_tfidf, newy, x_test_tfidf, y_test,"linear_Regression_tfidf_sampled_.model")
lassoRegression(X_tfidf, newy, x_test_tfidf, y_test,"lasso_Regression_tfidf_sampled.model")
RidgeRegression(X_tfidf, newy, x_test_tfidf, y_test,"Ridge_Regression_tfidf_sampled.model")


Mean Square Error for Linear Regression is :  0.04531986239065142
Mean Square Error for Lasso Regression is :  0.0833801457354903
Mean Square Error for Ridge Regression is :  0.03101395869198319


In [ ]:
RandomForest(X_tfidf, newy, x_test_tfidf, y_test,"RandomForest_Regression_tfidf_sampled.model")

In [41]:
linearRegression(x_train_word, newy, x_test_word, y_test,"linear_Regression_Word_2_Vec.model")
lassoRegression(x_train_word, newy, x_test_word, y_test,"lasso_Regression_Word_2_Vec.model")
RidgeRegression(x_train_word, newy, x_test_word, y_test,"Ridge_Regression_Word_2_Vec.model")


Mean Square Error for Linear Regression is :  0.06038906221356296
Mean Square Error for Lasso Regression is :  0.08338015363432451
Mean Square Error for Ridge Regression is :  0.060389080903820495


In [ ]:
RandomForest(x_train_word, newy, x_test_word, y_test,"RandomForest_Regression_Word_2_Vec.model")